In [ ]:
import pandas as pd

In [ ]:
times_data = pd.read_csv('progetto/timesData.csv')
times_data.head()

In [ ]:
shanghai_data = pd.read_csv('progetto/shanghaiData.csv')
shanghai_data.head()

In [ ]:
school_country = pd.read_csv('progetto/school_and_country_table.csv')
school_country.head()

In [ ]:
educational_attainment = pd.read_csv('progetto/educational_attainment_supplementary_data.csv')
educational_attainment.head()

In [ ]:
education_expenditure = pd.read_csv('progetto/education_expenditure_supplementary_data.csv')
education_expenditure.head()

In [ ]:
cwur = pd.read_csv('progetto/cwurData.csv')
cwur= cwur.rename(columns={'institution': 'university_name'})
cwur.head()

## 1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes

In [ ]:
times_data

In [ ]:
least_recent = times_data.iloc[times_data.groupby('university_name')['year'].idxmin()]

In [ ]:
most_recent = times_data.iloc[times_data.groupby('university_name')['year'].idxmax()]
most_recent

## 2. For each university, compute the improvement in income between the least recent and the most recent data points

In [ ]:
least_most_recent_join= pd.merge(least_recent[["university_name","year","income"]], most_recent[["university_name","year","income"]], on='university_name')
least_most_recent_join

In [ ]:
least_most_recent_join_v2=least_most_recent_join[(least_most_recent_join['year_x']!=least_most_recent_join['year_y']) & (least_most_recent_join['income_x'] !='-' ) & (least_most_recent_join['income_y'] !='-' )]


least_most_recent_join_v2['improvement']= pd.to_numeric(least_most_recent_join_v2['income_y'])- pd.to_numeric(least_most_recent_join_v2['income_x'])
least_most_recent_join_v2

## 3. Find the university with the largest increase computed in the previous point

In [ ]:
least_most_recent_join_v2.loc[least_most_recent_join_v2['improvement'].idxmax()]

In [ ]:
least_most_recent_join_v2.sort_values(by='improvement', ascending=False).head()

Utilizzando due metodi diversi otteniamo lo stesso risultato: sia prendendo il max con idxmax sia ordinando in maniera decrescente, notiamo che l'università con l'improvement di income maggiore è **TU Dresden**, che presenta un miglioramento di **67.8**

## 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

### da rivedere
non funziona perchè ci sono rank espressi come range.. bisogna parlarne con i ragazzi e trovare una soluzione

In [ ]:
cwur.head()

In [ ]:
shanghai_data.head()

In [ ]:
times_most_recent=times_data.loc[times_data.groupby(["university_name"])["year"].idxmax()]
cwur_most_recent=cwur.loc[cwur.groupby(["university_name"])["year"].idxmax()]
shanghai_most_recent=shanghai_data.loc[shanghai_data.groupby(["university_name"])["year"].idxmax()]

In [ ]:
times_most_recent_1= times_most_recent[['university_name','year','world_rank']]
cwur_most_recent_1= cwur_most_recent[['university_name','year','world_rank']]
shanghai_most_recent_1=shanghai_most_recent[['university_name','year','world_rank']]

In [ ]:
rank=pd.merge(times_most_recent_1, cwur_most_recent_1, on ='university_name', how='outer')
total_rank= pd.merge(rank, shanghai_most_recent_1, on ='university_name', how='outer')
total_rank

In [ ]:
total_rank['max_rank']= total_rank[['world_rank', 'world_rank_x', 'world_rank_y']].max(axis=1)
total_rank['min_rank']= total_rank[['world_rank', 'world_rank_x', 'world_rank_y']].min(axis=1)
total_rank['maximum_difference']= (total_rank['max_rank'] - total_rank['max_rank'])

In [ ]:
total_rank.sort_values(by='maximum_difference', ascending=False)

## 5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

In [ ]:
import re

In [84]:
most_recent.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016


In [ ]:
# espressione regolare per i casi 'buoni'. 
# ho creato la funzione da applicare successivamente in cui metto a 0 il campo se c'è il trattino ma c'è qualcosa che non va
male_female_re=re.compile('(?P<female>\d*)(\s:\s)(?P<male>\d*)')

def female(elem):
    occ = male_female_re.search(elem)
    if occ.match== False :
        female = 0
    else:
        female= int(occ.group('female'))
    return female

def male(elem):
    occ = male_female_re.search(elem)
    male= int(occ.group('male'))
    return male

In [ ]:
#applico la funzione in modo tale da creare una colonna per le femmine e una per i maschi 
# cosi poi da poter ottenere il numero di maschi e il numero di femmine per ogni nazione
most_recent['male'] = most_recent['female_male_ratio'].apply(male)
most_recent['female'] = most_recent['female_male_ratio'].apply(female)
